<a href="https://colab.research.google.com/github/AparnaMadva/Miscellaneous/blob/main/TopicModeling_BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langdetect indic-nlp-library bertopic
!pip install torch transformers accelerate sentence_transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
import string
import pandas as pd

In [ ]:
import datasets
from datasets import load_dataset
import transformers
from transformers.pipelines import pipeline

import langdetect
from indicnlp.tokenize import indic_tokenize, sentence_tokenize

### Load dataset

In [ ]:
formal_ds = load_dataset("wikimedia/wikipedia", "20231101.kn", split="train[:500]")
formal_ds = formal_ds.remove_columns(['id','title','url'])
formal_ds

Dataset({
    features: ['text'],
    num_rows: 100
})

In [ ]:
formal_df = formal_ds.to_pandas()
formal_df[:5]

,text
0,ಶಿವಮೊಗ್ಗ ಭಾರತ ದೇಶದ ಕರ್ನಾಟಕ ರಾಜ್ಯದ ಒಂದು ನಗರ (ಸಂ...
1,"ಭಾರತ, ಅಧಿಕೃತವಾಗಿ ಭಾರತ ಗಣರಾಜ್ಯ, ಅಥವಾ ಇಂಡಿಯಾ (In..."
2,ಜೋಗ ಅಥವಾ 'ಗೇರುಸೊಪ್ಪಿನ ಜಲಪಾತ' ಪ್ರಪಂಚದ ಪ್ರಸಿದ್ಧ ...
3,"ಕುವೆಂಪು, ಕುಪ್ಪಳಿ ವೆಂಕಟಪ್ಪ ಪುಟ್ಟಪ್ಪ (ಡಿಸೆಂಬರ್ ೨..."
4,"\n\nಟಿ.ಪಿ.ಕೈಲಾಸಂ\n""ನಕ್ಕು ನಗಿಸುವಾತ ಸಾವಿರ್ಜನಕ್ತ್..."


### Pre-processing

In [ ]:
# to-do: the below code snippet can remove isolated English text only and throws langdetect error for words seperated by '\n' such as 'ಗೊತ್ತಾ\nಪ್ರಮುಖ'
# ps. trivial tokenize function doesn't separate these two words and they have to be separated manually.

In [ ]:
cleaned_doc = pd.DataFrame()

exclude = set(string.punctuation)
exclude.update({"?","|"})

for text in formal_df['text']:
  tokens = indic_tokenize.trivial_tokenize(text)
  new_text = ""
  for t in tokens:
    if t not in exclude or t == " ":
      if langdetect.detect(t) == 'kn':

        new_text = new_text + t
  temp = pd.DataFrame([new_text])
  cleaned_doc = pd.concat([cleaned_doc,temp])

### Topic modeling using BERTopic

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

In [ ]:
# Create embeddings
docs = formal_df['text']
sentence_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)

# Create topic model
topic_model = BERTopic().fit(docs, embeddings)
topics, probs = topic_model.transform(docs, embeddings)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
topic_model.get_topic_freq().head()

,Topic,Count
1,0,55
0,-1,34
2,1,11


In [ ]:
# gives out only 3 topics due to the presence of commonly used English phrases in the dataset
print(set(topics))

{0, 1, -1}


In [ ]:
topic_model.get_topic(1)

[('the', 0.0636725678876057),
 ('1947', 0.05645093392888761),
 ('of', 0.043313733187805244),
 ('1965', 0.03316942589216207),
 ('15', 0.031096336773901938),
 ('14', 0.030644653369135043),
 ('22', 0.029472499303140567),
 ('2011', 0.029472499303140567),
 ('and', 0.029327241087525702),
 ('school', 0.028293746169988296)]

In [ ]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,"ಸದ್ಯದಲ್ಲೇ ಬರಲಿದೆ ಒಂದು ರುಪಾಯಿ ಹೊಸ ನೋಟು, ಏನೆಲ್ಲಾ...",-1,-1_var_else_googletagcmdpushfunction_settimeou...,"[var, else, googletagcmdpushfunction, settimeo...",[ಉತ್ತಮ ಫಲಿತಾಂಶಕ್ಕಾಗಿ ಪರಿಶ್ರಮ ಅಗತ್ಯ: ಡಾ.ವಿ.ರಾಮ್...,var - else - googletagcmdpushfunction - settim...,0.000000,False
1,ಬಿಜೆಪಿ ಅಧಿಕಾರಕ್ಕೆ ಬಂದರೂ ನಿಂತಿಲ್ಲ ಆಪರೇಷನ್ ಕಮಲ? ...,0,0_var_else_ist_kannada,"[var, else, ist, kannada, googletagcmdpushfunc...",[ಹೂಳು ತುಂಬಿದ ಕೆರೆಗಳಿಗೆ ನದಿ ನೀರು | Prajavani\nಹ...,var - else - ist - kannada - googletagcmdpushf...,0.954034,False
2,"ಸೆನೆಗಲ್ ಫುಟ್ಬಾಲ್ ಫೈನಲ್ ಪಂದ್ಯದ ವೇಳೆ ಘರ್ಷಣೆ, ಗೋಡ...",0,0_var_else_ist_kannada,"[var, else, ist, kannada, googletagcmdpushfunc...",[ಹೂಳು ತುಂಬಿದ ಕೆರೆಗಳಿಗೆ ನದಿ ನೀರು | Prajavani\nಹ...,var - else - ist - kannada - googletagcmdpushf...,0.875257,False
3,"ಕೋವಿಡ್‌ ಹೊಡೆತಕ್ಕೆ ಐಸ್ ಕ್ರೀಂ, ಜ್ಯೂಸ್ ಪಾರ್ಲರ್‌ಗಳ...",-1,-1_var_else_googletagcmdpushfunction_settimeou...,"[var, else, googletagcmdpushfunction, settimeo...",[ಉತ್ತಮ ಫಲಿತಾಂಶಕ್ಕಾಗಿ ಪರಿಶ್ರಮ ಅಗತ್ಯ: ಡಾ.ವಿ.ರಾಮ್...,var - else - googletagcmdpushfunction - settim...,0.000000,False
4,ಕವಿತೆ | ಮನದಲೆ ಬೆಂದು ಬಡವಾದೆ ಏಕೆ? - Kannada news...,0,0_var_else_ist_kannada,"[var, else, ist, kannada, googletagcmdpushfunc...",[ಹೂಳು ತುಂಬಿದ ಕೆರೆಗಳಿಗೆ ನದಿ ನೀರು | Prajavani\nಹ...,var - else - ist - kannada - googletagcmdpushf...,0.885047,False
...,...,...,...,...,...,...,...,...
95,‘ಕೊರೊನಾ ಸುರಕ್ಷಾ ಕವಚ’ ಕೊರತೆ!? | Prajavani\n‘ಕೊರ...,0,0_var_else_ist_kannada,"[var, else, ist, kannada, googletagcmdpushfunc...",[ಹೂಳು ತುಂಬಿದ ಕೆರೆಗಳಿಗೆ ನದಿ ನೀರು | Prajavani\nಹ...,var - else - ist - kannada - googletagcmdpushf...,0.859792,True
96,ಸ್ಪಿರೋ ಕೀಬೋರ್ಡ್ಸ್ ಬೆಲೆ India ಇನ್ 17 Nov 2019 ರ...,-1,-1_var_else_googletagcmdpushfunction_settimeou...,"[var, else, googletagcmdpushfunction, settimeo...",[ಉತ್ತಮ ಫಲಿತಾಂಶಕ್ಕಾಗಿ ಪರಿಶ್ರಮ ಅಗತ್ಯ: ಡಾ.ವಿ.ರಾಮ್...,var - else - googletagcmdpushfunction - settim...,0.000000,False
97,ಕಸ ವಿಲೇವಾರಿಗೆ ಇಂದೋರ್‌ ಮಾದರಿ | Udayavani – ಉದಯವ...,-1,-1_var_else_googletagcmdpushfunction_settimeou...,"[var, else, googletagcmdpushfunction, settimeo...",[ಉತ್ತಮ ಫಲಿತಾಂಶಕ್ಕಾಗಿ ಪರಿಶ್ರಮ ಅಗತ್ಯ: ಡಾ.ವಿ.ರಾಮ್...,var - else - googletagcmdpushfunction - settim...,0.000000,False
98,ಕಳಸ: ಕಂದಕಕ್ಕೆ ಉರುಳಿದ ಕಾರು; ಇಬ್ಬರು ಸಹೋದರ ದಂಪತಿಯ...,0,0_var_else_ist_kannada,"[var, else, ist, kannada, googletagcmdpushfunc...",[ಹೂಳು ತುಂಬಿದ ಕೆರೆಗಳಿಗೆ ನದಿ ನೀರು | Prajavani\nಹ...,var - else - ist - kannada - googletagcmdpushf...,0.914938,False


In [ ]:
topic_model.visualize_topics()

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_heatmap()

Also see :
[Topics per Class Using BERTopic](https://towardsdatascience.com/topics-per-class-using-bertopic-252314f2640)